## _Drawing Functions_

- _Detector Layout_
- _Event Plotting (CSVs, Torch)_
- _A.O.B._

In [ ]:
import glob, os, sys, yaml

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pprint
import seaborn as sns
import trackml.dataset

In [ ]:
import torch
from torch_geometric.data import Data
import itertools

In [ ]:
# append parent dir
sys.path.append('..')

In [ ]:
# get cuda device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# local imports
from src import SttCSVDataReader, SttTorchDataReader
from src import Visualize_CSVEvent, Visualize_TorchEvent
from src import detector_layout

### _Load Data_

In [ ]:
# mu+mu- data (current)
input_dir = '../data_all'

In [ ]:
# Find All Input Data Files (hits.csv, cells.csv, particles.csv, truth.csv)
all_files = os.listdir(input_dir)

# Extract File Prefixes (use e.g. xxx-hits.csv)
suffix = '-hits.csv'
file_prefixes = sorted(os.path.join(input_dir, f.replace(suffix, ''))
                       for f in all_files if f.endswith(suffix))

In [ ]:
file_prefixes[:10]

In [ ]:
event_id = 1
event_prefix = file_prefixes[event_id]

In [ ]:
# load an event
hits, tubes, particles, truth = trackml.dataset.load_event(event_prefix)

# memory usage
mem_bytes = (hits.memory_usage(index=True).sum() 
             + tubes.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())

print('{} memory usage {:.2f} MB'.format(os.path.basename(event_prefix), mem_bytes / 2**20))

In [ ]:
hits.head()

In [ ]:
tubes.head()

In [ ]:
particles.head()

In [ ]:
truth.head()

### _Detector Layout_

In [ ]:
detector_layout(figsize=(10,10));

### _SttCSVDataReader_

In [ ]:
# use SttCSVDataReader
inputdir = '../data_all'
csv_reader = SttCSVDataReader(inputdir,noise=False,skewed=False,selection=False)
data = csv_reader(evtid=event_id)
event = data.event

In [ ]:
# visualize CSVEvent
Visualize_CSVEvent(event,figsize=(10,10),save_fig=False);

### _SttTorchDataReader_

In [ ]:
# use SttTorchDataReader
inputdir="../run/feature_store"
torch_reader = SttTorchDataReader(inputdir)
feature_data = torch_reader(evtid=event_id)

In [ ]:
Visualize_TorchEvent(feature_data,figsize=(10,10),save_fig=False);

### _Plot True & False Edges_

In [ ]:
from src import detector_layout
from src.math_utils import polar_to_cartesian

In [ ]:
event_idx = 1

In [ ]:
inputdir="../run/feature_store"
proc_files = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
# event_id
proc_files[event_idx]

In [ ]:
feature_data = torch.load(proc_files[event_idx], map_location=device)

In [ ]:
# get spatial coordinates, note: x = [x,y,ir]
r, phi, ir = feature_data.x.T

In [ ]:
# coord. transform, compensate scaling
x, y = polar_to_cartesian(r, phi)

In [ ]:
# plot all hits
fig, ax = detector_layout(figsize=(10,10))
ax.scatter(x, y, s=20)

ax.set_title("Azimuthal View of Detector", fontsize=15)
ax.grid(False)
fig.tight_layout()
# fig.savefig(f'event_{event_idx}.png')

In [ ]:
# get true_edges
e = feature_data.edge_index
pid = feature_data.pid
true_edges = pid[e[0]] == pid[e[1]]

In [ ]:
# plot only true edges
fig, ax = detector_layout(figsize=(10,10))

ax.plot(x[e[:, true_edges]], y[e[:, true_edges]], c="b")
ax.scatter(x, y, s=20)

ax.set_title("Azimuthal View of Detector", fontsize=15)
ax.grid(False)
fig.tight_layout()
# fig.savefig(f'true_edges_{event_idx}.png')

In [ ]:
# plot only false edges
fig, ax = detector_layout(figsize=(10,10))

ax.plot(x[e[:, ~true_edges]], y[e[:, ~true_edges]], c="r")
ax.scatter(x, y, s=20)

ax.set_title("Azimuthal View of Detector", fontsize=15)
ax.grid(False)
fig.tight_layout()
# fig.savefig(f'false_edges_{event_idx}.png')

In [ ]:
# plot all edges (true + false), only few false edges
fig, ax = detector_layout(figsize=(10,10))

ax.plot(x[e[:, (~true_edges)][:, 0:-1:5]], y[e[:, (~true_edges)][:, 0:-1:5]], c="r")
ax.scatter(x, y, s=20)

ax.set_title("Azimuthal View of Detector", fontsize=15)
ax.grid(False)
fig.tight_layout()
# fig.savefig(f'true_edges_less_{event_idx}.png')

## _Inspect HDFStore_

- See [IO tools (text, CSV, HDF5, …)](https://pandas.pydata.org/docs/user_guide/io.html)

#### HDF5 Store

- read/write using `pd.HDFStore("path/to/*.h5")`
- read/write using `pd.read_hdf()`, `pd.to_hdf()` similar to `pd.read_csv()` and `pd.to_csv()`
    *. `pd.read_hdf()`: Read from the store, close it if we opened it.
- close when finished